# NYT Connections solver

### utils functionality demo

In [1]:
from scripts.puzzle_utils import Puzzle
from scripts.solver import Solver

puzzle = Puzzle()
random_solver = Solver()

# retrieve random puzzle from the database
random_words, id = puzzle.get_random_puzzle()
print(random_words)

# attempt to solve it (the example here is random ordering of words)
random_solver.solve_puzzle(random_words)
attempt_sol = random_solver.build_solution(random_words)
print(attempt_sol)

# check the solution
is_sol, num_corr = puzzle.check_solution(attempt_sol, id)
print(f"Is solution? {is_sol} \nNumber of matches: {num_corr}")

['NOSE', 'HEAD', 'CRAYON', 'BULB', 'HONEYCOMB', 'SEAL', 'STALK', 'CABIN', 'ROB', 'WING', 'ENGINE', 'FLEECE', 'CANDLE', 'EAR', 'HOSE', 'STIFF']
[{'CANDLE', 'SEAL', 'NOSE', 'WING'}, {'FLEECE', 'EAR', 'HOSE', 'STIFF'}, {'HEAD', 'ENGINE', 'STALK', 'HONEYCOMB'}, {'CRAYON', 'CABIN', 'BULB', 'ROB'}]
Is solution? False 
Number of matches: 7


In [2]:
from scripts.glove import GloVe

glove = GloVe("embeddings/glove.6B.300d.txt")

Creating GloVe lookup map: 400000it [00:08, 47823.02it/s]


In [3]:
import numpy as np
print(glove.embed_puzzle_words(random_words))

[[-0.18221   0.23007  -0.35024  ...  0.020427  0.17435   0.10724 ]
 [-0.27105   0.10095  -0.033651 ...  0.46617  -0.12275   0.31105 ]
 [-0.057725  0.27479   0.17358  ...  0.15228   0.21541   0.047977]
 ...
 [-0.43982  -0.20805  -0.80559  ...  0.53964   0.21984   0.098529]
 [ 0.39486  -0.41406   0.062148 ... -0.14045   0.3879    0.60855 ]
 [-0.38626  -0.36741  -0.32377  ...  0.45258   0.19781  -0.14027 ]]


### k-means approach

In [4]:
from scripts.kmeans import KMeansSolver

# retrieve random puzzle from the database
random_words, id = puzzle.get_random_puzzle()
print(random_words)

# attempt to solve it (fixed size kmeans)
kmeans_solver = KMeansSolver(glove)
kmeans_solver.solve_puzzle(random_words)
attempt_sol = kmeans_solver.build_solution(random_words)
print(attempt_sol)

# check the solution
is_sol, num_corr = puzzle.check_solution(attempt_sol, id)
print(f"Is solution? {is_sol} \nNumber of matches: {num_corr}")

['GIFT', 'AMAZON', 'YELLOW', 'APPLE', 'LIME', 'RHINE', 'INTUIT', 'INSTINCT', 'ORACLE', 'SURFACE', 'NOSE', 'FLAIR', 'FRONT', 'BRIM', 'FACE', 'OUTSIDE']
[{'BRIM', 'RHINE', 'NOSE', 'SURFACE'}, {'YELLOW', 'OUTSIDE', 'FRONT', 'FACE'}, {'ORACLE', 'AMAZON', 'APPLE', 'INTUIT'}, {'INSTINCT', 'LIME', 'GIFT', 'FLAIR'}]
Is solution? False 
Number of matches: 10


### neural approach

In [6]:
from scripts.neural.dataset import ConnectionsDataset

dataset = ConnectionsDataset(glove, puzzle)

creatiing csv: 100%|██████████| 500/500 [00:00<00:00, 2166.04it/s]


### testing approaches

In [15]:
from scripts.tester import MethodTester

tester = MethodTester()
results_random = tester.test_solver(random_solver)
results_kmeans = tester.test_solver(kmeans_solver)

print(f"Random Solver:\nNumber of solutions: {np.sum(results_random['is_sol'])}/{len(results_random['is_sol'])}\nAverage number of correct words: {np.mean(results_random['num_corr'])}/16")
# currently not all puzzles are solved with kmeans to do some phrases not having an embeddings
# this mostly happens for complex phrases like 'jack in the box'
# work in progress on that one
print(f"K-means Solver:\nNumber of solutions: {np.sum(results_kmeans['is_sol'])}/{len(results_kmeans['is_sol'])}\nAverage number of correct words: {np.mean(results_kmeans['num_corr'])}/16")

100%|██████████| 645/645 [00:00<00:00, 1810.11it/s]

Random Solver:
Number of solutions: 0/645
Average number of correct words: 5.6682170542635655/16
K-means Solver:
Number of solutions: 9/642
Average number of correct words: 7.750778816199377/16
